In [67]:
import os
import shutil
import string
import json
import webvtt as wv
import subprocess as sp
from pydub import AudioSegment

In [76]:
url = "https://www.youtube.com/watch?v=1KTRkRQYKPQ"

In [86]:
os.path.splitext("a.en.vtt")

('a.en', '.vtt')

In [79]:
cmd = f"youtube-dl --print-json --write-auto-sub -x --audio-format wav {url}"
p = sp.Popen(cmd.split(), stdout=sp.PIPE, stderr=sp.PIPE, cwd="data")
out, err = p.communicate()
out = json.loads(out)
title = out['title'].split()
season = title[title.index("Season")+1]
episode = title[title.index("Episode")+1]
old_fn = os.path.splitext(out['_filename'])[0]
new_fn = f"s{season}e{episode}"
data_dir = os.path.join("data", new_fn)
prefix = os.path.join(data_dir, new_fn)
os.makedirs(data_dir, exist_ok=True)
os.rename(os.path.join("data", old_fn+".wav"), prefix+".wav")
os.rename(os.path.join("data", old_fn+".en.vtt"), prefix+".en.vtt")
chunk_dir = os.path.join(data_dir, "chunks")

In [80]:
wav = AudioSegment.from_wav(prefix+'.wav')

In [82]:
if os.path.exists(chunk_dir):
    shutil.rmtree(chunk_dir)
os.makedirs(chunk_dir, exist_ok=True)
punc_table = str.maketrans('', '', string.punctuation)
text = ""
delta = 0
i = 0
for cap in wv.read(prefix+".en.vtt"):
    if len(text) == 0:
        start = cap.start_in_seconds
    next_text = cap.text.split('\n')[1]
    next_text = next_text.translate(punc_table)
    next_text = next_text.lower()
    text += next_text
    end = cap.end_in_seconds
    if end-start > 10:
        isalpha = True
        for sent in text.split('\n'):
            for word in text.split():
                isalpha = isalpha and word.isalpha()
        if isalpha:
            chunk_prefix = os.path.join(chunk_dir, f"chunk_{i}")
            wav[start*1e3:end*1e3].export(chunk_prefix+".wav", format="wav")
            with open(chunk_prefix+".txt", 'w') as fh:
                fh.write(text.strip())
            i += 1
        text = ""